In [ ]:
import polars as pl
import seaborn as sb
import gtfs_delay_analysis as da 

In [ ]:
import importlib
importlib.reload(da)

In [ ]:
aggregated = da.load_aggregate_data()
stops = pl.read_parquet('stops.parquet')
raw_dfs = da.load_raw_data()

In [ ]:
stops.write_parquet('stops.parquet')

In [ ]:
trips = da.trips.load_trips_without_shapes_df()
str_shapes = da.trips.load_str_shapes_df()
shapes = da.trips.load_parsed_shapes_df()
network_so_bad = pl.read_parquet('sequence.parquet')

In [ ]:
from gtfs_delay_analysis.ddelay import get_ddelay
with_shape = network_so_bad.pipe(get_ddelay).with_columns(
    pl.col('trip_id').cast(pl.Int64)
).join(trips.select('trip_id', 'shape_id'), on='trip_id')

In [ ]:
with_line = with_shape.join(shapes, on='shape_id').drop('shape_id', 'line_length', 'line')

In [ ]:
with_id = (with_line
 .with_row_index('id')
 .with_columns(pl.col('id').floordiv(50000)))

In [ ]:
split = with_id.partition_by('id')

In [ ]:
for s, df in enumerate(split):
    df.write_parquet(f'with-shape-{s}.parquet')